In [1]:
using JuMP, Cbc, GLPK

In [2]:
model = Model(with_optimizer(GLPK.Optimizer))

# Number of columns, rows, values

@variable(model, x[i=1:9, j=1:9, k=1:9], Bin)
# The syntax of this model is such that if any indices of x has a number k it will equal 1

9×9×9 Array{VariableRef,3}:
[:, :, 1] =
 x[1,1,1]  x[1,2,1]  x[1,3,1]  x[1,4,1]  …  x[1,7,1]  x[1,8,1]  x[1,9,1]
 x[2,1,1]  x[2,2,1]  x[2,3,1]  x[2,4,1]     x[2,7,1]  x[2,8,1]  x[2,9,1]
 x[3,1,1]  x[3,2,1]  x[3,3,1]  x[3,4,1]     x[3,7,1]  x[3,8,1]  x[3,9,1]
 x[4,1,1]  x[4,2,1]  x[4,3,1]  x[4,4,1]     x[4,7,1]  x[4,8,1]  x[4,9,1]
 x[5,1,1]  x[5,2,1]  x[5,3,1]  x[5,4,1]     x[5,7,1]  x[5,8,1]  x[5,9,1]
 x[6,1,1]  x[6,2,1]  x[6,3,1]  x[6,4,1]  …  x[6,7,1]  x[6,8,1]  x[6,9,1]
 x[7,1,1]  x[7,2,1]  x[7,3,1]  x[7,4,1]     x[7,7,1]  x[7,8,1]  x[7,9,1]
 x[8,1,1]  x[8,2,1]  x[8,3,1]  x[8,4,1]     x[8,7,1]  x[8,8,1]  x[8,9,1]
 x[9,1,1]  x[9,2,1]  x[9,3,1]  x[9,4,1]     x[9,7,1]  x[9,8,1]  x[9,9,1]

[:, :, 2] =
 x[1,1,2]  x[1,2,2]  x[1,3,2]  x[1,4,2]  …  x[1,7,2]  x[1,8,2]  x[1,9,2]
 x[2,1,2]  x[2,2,2]  x[2,3,2]  x[2,4,2]     x[2,7,2]  x[2,8,2]  x[2,9,2]
 x[3,1,2]  x[3,2,2]  x[3,3,2]  x[3,4,2]     x[3,7,2]  x[3,8,2]  x[3,9,2]
 x[4,1,2]  x[4,2,2]  x[4,3,2]  x[4,4,2]     x[4,7,2]  x[4,8,2]  x[4,9,2

For these next few constraints this took time even with presented solutions to the sudoku. The syntax for these constraints were constraints were added such that they fit the true formula. For example, the column constraints were the sum of all the i's from 1 to 9 **(aka the rows)**.

So mathematically the individual column required a (n_rows x n_possible_k's). This means that for every individual column sum we will have 9x9 constriants. So the syntax is written in the form such that **every  row for every column can have ONE possible k which is why the SUM will be one**. Only one k at a specific x[i,j,k] will be 1 while all others of that one will be off. Once it is it will be selected for in a solution parser function below.

# Column Constraint

The syntax of this constraint is

@constraint ( name of Model(),

individual name of constraints (which i iterated) ,

expression i wanted my constraint to be (i.e. the formula from class)
)

In [3]:
@constraint(model, col[j=1:9,k=1:9], sum(x[i,j,k] for i in 1:9 )==1)

9×9 Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,2}:
 col[1,1] : x[1,1,1] + x[2,1,1] + x[3,1,1] + x[4,1,1] + x[5,1,1] + x[6,1,1] + x[7,1,1] + x[8,1,1] + x[9,1,1] == 1.0  …  col[1,9] : x[1,1,9] + x[2,1,9] + x[3,1,9] + x[4,1,9] + x[5,1,9] + x[6,1,9] + x[7,1,9] + x[8,1,9] + x[9,1,9] == 1.0
 col[2,1] : x[1,2,1] + x[2,2,1] + x[3,2,1] + x[4,2,1] + x[5,2,1] + x[6,2,1] + x[7,2,1] + x[8,2,1] + x[9,2,1] == 1.0     col[2,9] : x[1,2,9] + x[2,2,9] + x[3,2,9] + x[4,2,9] + x[5,2,9] + x[6,2,9] + x[7,2,9] + x[8,2,9] + x[9,2,9] == 1.0
 col[3,1] : x[1,3,1] + x[2,3,1] + x[3,3,1] + x[4,3,1] + x[5,3,1] + x[6,3,1] + x[7,3,1] + x[8,3,1] + x[9,3,1] == 1.0     col[3,9] : x[1,3,9] + x[2,3,9] + x[3,3,9] + x[4,3,9] + x[5,3,9] + x[6,3,9] + x[7,3,9] + x[8,3,9] + x[9,3,9] == 1.0
 col[4,1] : x[1,4,1] + x[2,4,1] + x[3,4,1] + x[4,4,1] + x[5,4,1] + x[6,4,1] + x[7,4,1] + x[8,4,1] + x[9,4,1] == 1.0     col[4,9] : x[1,4,9] + x[2,4,9] + x[3,4,9] + x[4,4,9] + x[5,4,9] + x[6,4,9] + x[7,4,9] + x[8,4,9] 

# Row Constraint

In [4]:
@constraint(model,row[i=1:9,k=1:9], sum(x[i,j,k] for j in 1:9 )==1)

9×9 Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,2}:
 row[1,1] : x[1,1,1] + x[1,2,1] + x[1,3,1] + x[1,4,1] + x[1,5,1] + x[1,6,1] + x[1,7,1] + x[1,8,1] + x[1,9,1] == 1.0  …  row[1,9] : x[1,1,9] + x[1,2,9] + x[1,3,9] + x[1,4,9] + x[1,5,9] + x[1,6,9] + x[1,7,9] + x[1,8,9] + x[1,9,9] == 1.0
 row[2,1] : x[2,1,1] + x[2,2,1] + x[2,3,1] + x[2,4,1] + x[2,5,1] + x[2,6,1] + x[2,7,1] + x[2,8,1] + x[2,9,1] == 1.0     row[2,9] : x[2,1,9] + x[2,2,9] + x[2,3,9] + x[2,4,9] + x[2,5,9] + x[2,6,9] + x[2,7,9] + x[2,8,9] + x[2,9,9] == 1.0
 row[3,1] : x[3,1,1] + x[3,2,1] + x[3,3,1] + x[3,4,1] + x[3,5,1] + x[3,6,1] + x[3,7,1] + x[3,8,1] + x[3,9,1] == 1.0     row[3,9] : x[3,1,9] + x[3,2,9] + x[3,3,9] + x[3,4,9] + x[3,5,9] + x[3,6,9] + x[3,7,9] + x[3,8,9] + x[3,9,9] == 1.0
 row[4,1] : x[4,1,1] + x[4,2,1] + x[4,3,1] + x[4,4,1] + x[4,5,1] + x[4,6,1] + x[4,7,1] + x[4,8,1] + x[4,9,1] == 1.0     row[4,9] : x[4,1,9] + x[4,2,9] + x[4,3,9] + x[4,4,9] + x[4,5,9] + x[4,6,9] + x[4,7,9] + x[4,8,9] 

# Completely Filled Sudoku Constraint

In [5]:
@constraint(model, filled[i=1:9,j=1:9], sum(x[i,j,k] for k in 1:9)==1)

9×9 Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,2}:
 filled[1,1] : x[1,1,1] + x[1,1,2] + x[1,1,3] + x[1,1,4] + x[1,1,5] + x[1,1,6] + x[1,1,7] + x[1,1,8] + x[1,1,9] == 1.0  …  filled[1,9] : x[1,9,1] + x[1,9,2] + x[1,9,3] + x[1,9,4] + x[1,9,5] + x[1,9,6] + x[1,9,7] + x[1,9,8] + x[1,9,9] == 1.0
 filled[2,1] : x[2,1,1] + x[2,1,2] + x[2,1,3] + x[2,1,4] + x[2,1,5] + x[2,1,6] + x[2,1,7] + x[2,1,8] + x[2,1,9] == 1.0     filled[2,9] : x[2,9,1] + x[2,9,2] + x[2,9,3] + x[2,9,4] + x[2,9,5] + x[2,9,6] + x[2,9,7] + x[2,9,8] + x[2,9,9] == 1.0
 filled[3,1] : x[3,1,1] + x[3,1,2] + x[3,1,3] + x[3,1,4] + x[3,1,5] + x[3,1,6] + x[3,1,7] + x[3,1,8] + x[3,1,9] == 1.0     filled[3,9] : x[3,9,1] + x[3,9,2] + x[3,9,3] + x[3,9,4] + x[3,9,5] + x[3,9,6] + x[3,9,7] + x[3,9,8] + x[3,9,9] == 1.0
 filled[4,1] : x[4,1,1] + x[4,1,2] + x[4,1,3] + x[4,1,4] + x[4,1,5] + x[4,1,6] + x[4,1,7] + x[4,1,8] + x[4,1,9] == 1.0     filled[4,9] : x[4,9,1] + x[4,9,2] + x[4,9,3] + x[4,9,4] + x[4,9,5] + x[4,9,6

# Sub Matrices  Sudoku Constraint

In [6]:
@constraint(model, submatUL[k=1:9], sum(sum(x[1:3,1:3,k]))==1)
@constraint(model, submatUM[k=1:9], sum(sum(x[1:3,4:6,k]))==1)
@constraint(model, submatUR[k=1:9], sum(sum(x[1:3,7:9,k]))==1)

@constraint(model, submatML[k=1:9], sum(sum(x[4:6,1:3,k]))==1)
@constraint(model, submatMM[k=1:9], sum(sum(x[4:6,4:6,k]))==1)
@constraint(model, submatMR[k=1:9], sum(sum(x[4:6,7:9,k]))==1)

@constraint(model, submatLL[k=1:9], sum(sum(x[7:9,1:3,k]))==1)
@constraint(model, submatLM[k=1:9], sum(sum(x[7:9,4:6,k]))==1)
@constraint(model, submatLR[k=1:9], sum(sum(x[7:9,7:9,k]))==1)

9-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 submatLR[1] : x[7,7,1] + x[8,7,1] + x[9,7,1] + x[7,8,1] + x[8,8,1] + x[9,8,1] + x[7,9,1] + x[8,9,1] + x[9,9,1] == 1.0
 submatLR[2] : x[7,7,2] + x[8,7,2] + x[9,7,2] + x[7,8,2] + x[8,8,2] + x[9,8,2] + x[7,9,2] + x[8,9,2] + x[9,9,2] == 1.0
 submatLR[3] : x[7,7,3] + x[8,7,3] + x[9,7,3] + x[7,8,3] + x[8,8,3] + x[9,8,3] + x[7,9,3] + x[8,9,3] + x[9,9,3] == 1.0
 submatLR[4] : x[7,7,4] + x[8,7,4] + x[9,7,4] + x[7,8,4] + x[8,8,4] + x[9,8,4] + x[7,9,4] + x[8,9,4] + x[9,9,4] == 1.0
 submatLR[5] : x[7,7,5] + x[8,7,5] + x[9,7,5] + x[7,8,5] + x[8,8,5] + x[9,8,5] + x[7,9,5] + x[8,9,5] + x[9,9,5] == 1.0
 submatLR[6] : x[7,7,6] + x[8,7,6] + x[9,7,6] + x[7,8,6] + x[8,8,6] + x[9,8,6] + x[7,9,6] + x[8,9,6] + x[9,9,6] == 1.0
 submatLR[7] : x[7,7,7] + x[8,7,7] + x[9,7,7] + x[7,8,7] + x[8,8,7] + x[9,8,7] + x[7,9,7] + x[8,9,7] + x[9,9,7] == 1.0
 submatLR[8] : x[7,7,8] + x[8,7,8] + x[9,7,8] + x[7,8,8] + x[8,8,8] + x[9,8,8] + x[

# Diagonal Constraint

In [7]:
row_DT = []
for i in 9:-1:1
    append!(row_DT,i)
end


col_DT = []
for j in 1:9
    append!(col_DT,j)
end

@constraint(model, diagonalDT[k=1:9], sum(sum(x[row_DT[i],col_DT[i],k] for i in 1:9))==1)

9-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 diagonalDT[1] : x[9,1,1] + x[8,2,1] + x[7,3,1] + x[6,4,1] + x[5,5,1] + x[4,6,1] + x[3,7,1] + x[2,8,1] + x[1,9,1] == 1.0
 diagonalDT[2] : x[9,1,2] + x[8,2,2] + x[7,3,2] + x[6,4,2] + x[5,5,2] + x[4,6,2] + x[3,7,2] + x[2,8,2] + x[1,9,2] == 1.0
 diagonalDT[3] : x[9,1,3] + x[8,2,3] + x[7,3,3] + x[6,4,3] + x[5,5,3] + x[4,6,3] + x[3,7,3] + x[2,8,3] + x[1,9,3] == 1.0
 diagonalDT[4] : x[9,1,4] + x[8,2,4] + x[7,3,4] + x[6,4,4] + x[5,5,4] + x[4,6,4] + x[3,7,4] + x[2,8,4] + x[1,9,4] == 1.0
 diagonalDT[5] : x[9,1,5] + x[8,2,5] + x[7,3,5] + x[6,4,5] + x[5,5,5] + x[4,6,5] + x[3,7,5] + x[2,8,5] + x[1,9,5] == 1.0
 diagonalDT[6] : x[9,1,6] + x[8,2,6] + x[7,3,6] + x[6,4,6] + x[5,5,6] + x[4,6,6] + x[3,7,6] + x[2,8,6] + x[1,9,6] == 1.0
 diagonalDT[7] : x[9,1,7] + x[8,2,7] + x[7,3,7] + x[6,4,7] + x[5,5,7] + x[4,6,7] + x[3,7,7] + x[2,8,7] + x[1,9,7] == 1.0
 diagonalDT[8] : x[9,1,8] + x[8,2,8] + x[7,3,8] + x[6,4,8] + x[5,5,8]

In [8]:
@constraint(model, diagonalTD[k=1:9], sum(sum(x[i,i,k] for i in 1:9))==1)

9-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 diagonalTD[1] : x[1,1,1] + x[2,2,1] + x[3,3,1] + x[4,4,1] + x[5,5,1] + x[6,6,1] + x[7,7,1] + x[8,8,1] + x[9,9,1] == 1.0
 diagonalTD[2] : x[1,1,2] + x[2,2,2] + x[3,3,2] + x[4,4,2] + x[5,5,2] + x[6,6,2] + x[7,7,2] + x[8,8,2] + x[9,9,2] == 1.0
 diagonalTD[3] : x[1,1,3] + x[2,2,3] + x[3,3,3] + x[4,4,3] + x[5,5,3] + x[6,6,3] + x[7,7,3] + x[8,8,3] + x[9,9,3] == 1.0
 diagonalTD[4] : x[1,1,4] + x[2,2,4] + x[3,3,4] + x[4,4,4] + x[5,5,4] + x[6,6,4] + x[7,7,4] + x[8,8,4] + x[9,9,4] == 1.0
 diagonalTD[5] : x[1,1,5] + x[2,2,5] + x[3,3,5] + x[4,4,5] + x[5,5,5] + x[6,6,5] + x[7,7,5] + x[8,8,5] + x[9,9,5] == 1.0
 diagonalTD[6] : x[1,1,6] + x[2,2,6] + x[3,3,6] + x[4,4,6] + x[5,5,6] + x[6,6,6] + x[7,7,6] + x[8,8,6] + x[9,9,6] == 1.0
 diagonalTD[7] : x[1,1,7] + x[2,2,7] + x[3,3,7] + x[4,4,7] + x[5,5,7] + x[6,6,7] + x[7,7,7] + x[8,8,7] + x[9,9,7] == 1.0
 diagonalTD[8] : x[1,1,8] + x[2,2,8] + x[3,3,8] + x[4,4,8] + x[5,5,8]

In [9]:
constraint_dict = model.obj_dict

Dict{Symbol,Any} with 15 entries:
  :submatUM   => ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where …
  :submatLR   => ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where …
  :submatLL   => ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where …
  :col        => ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where …
  :submatMM   => ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where …
  :x          => VariableRef[x[1,1,1] x[1,2,1] … x[1,8,1] x[1,9,1]; x[2,1,1] x[…
  :submatMR   => ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where …
  :diagonalDT => ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where …
  :submatUL   => ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where …
  :submatLM   => ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where …
  :row        => ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where …
  :diagonalTD => ConstraintRef{Model,C,Shape} where Shape<:AbstractShape wh

In [15]:
constraint_dict[:diagonalDT]

9-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 diagonalDT[1] : x[9,1,1] + x[8,2,1] + x[7,3,1] + x[6,4,1] + x[5,5,1] + x[4,6,1] + x[3,7,1] + x[2,8,1] + x[1,9,1] == 1.0
 diagonalDT[2] : x[9,1,2] + x[8,2,2] + x[7,3,2] + x[6,4,2] + x[5,5,2] + x[4,6,2] + x[3,7,2] + x[2,8,2] + x[1,9,2] == 1.0
 diagonalDT[3] : x[9,1,3] + x[8,2,3] + x[7,3,3] + x[6,4,3] + x[5,5,3] + x[4,6,3] + x[3,7,3] + x[2,8,3] + x[1,9,3] == 1.0
 diagonalDT[4] : x[9,1,4] + x[8,2,4] + x[7,3,4] + x[6,4,4] + x[5,5,4] + x[4,6,4] + x[3,7,4] + x[2,8,4] + x[1,9,4] == 1.0
 diagonalDT[5] : x[9,1,5] + x[8,2,5] + x[7,3,5] + x[6,4,5] + x[5,5,5] + x[4,6,5] + x[3,7,5] + x[2,8,5] + x[1,9,5] == 1.0
 diagonalDT[6] : x[9,1,6] + x[8,2,6] + x[7,3,6] + x[6,4,6] + x[5,5,6] + x[4,6,6] + x[3,7,6] + x[2,8,6] + x[1,9,6] == 1.0
 diagonalDT[7] : x[9,1,7] + x[8,2,7] + x[7,3,7] + x[6,4,7] + x[5,5,7] + x[4,6,7] + x[3,7,7] + x[2,8,7] + x[1,9,7] == 1.0
 diagonalDT[8] : x[9,1,8] + x[8,2,8] + x[7,3,8] + x[6,4,8] + x[5,5,8]

In [16]:
initial_grid = [
    3 0 0 0 0 0 0 0 9;
    0 0 0 9 0 0 0 7 5;
    0 0 0 0 0 0 0 0 0;
    0 0 4 8 0 6 0 0 2;
    5 0 0 1 0 0 0 0 0;
    8 0 6 0 3 0 4 5 0;
    0 0 8 0 0 0 0 0 0;
    0 0 0 0 0 0 0 0 0;
    0 0 0 0 0 0 0 3 0;
]

sol_grid = initial_grid[:,:]

9×9 Array{Int64,2}:
 3  0  0  0  0  0  0  0  9
 0  0  0  9  0  0  0  7  5
 0  0  0  0  0  0  0  0  0
 0  0  4  8  0  6  0  0  2
 5  0  0  1  0  0  0  0  0
 8  0  6  0  3  0  4  5  0
 0  0  8  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  3  0

# Tuning on the 'x[i,j,k]' values from the intial given soduko.

This cell below will take the intial solution from the matrix above and turn those specific x[i,j,k] constraints on **AS** constraints

In [17]:
for i=1:9
    for j=1:9
        if sol_grid[i,j]!=0
            @constraint(model, (x[i,j,sol_grid[i,j]])==1)
        end
    end
end

In [18]:
optimize!(model)

In [19]:
turned_on = JuMP.value.(x)
sol = zeros(Int,9,9)
for i in 1:9
    for j in 1:9
        for k in 1:9
            if turned_on[i,j,k]==1
                sol[i,j]=k
            end
        end
    end
end
sol

9×9 Array{Int64,2}:
 3  6  5  7  1  8  2  4  9
 4  2  1  9  6  3  8  7  5
 9  8  7  4  2  5  3  6  1
 7  3  4  8  5  6  1  9  2
 5  9  2  1  4  7  6  8  3
 8  1  6  2  3  9  4  5  7
 6  7  8  3  9  1  5  2  4
 2  5  3  6  7  4  9  1  8
 1  4  9  5  8  2  7  3  6